In [1]:
import requests
from requests import Response
url = "https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json"
try:
    response:Response = requests.get(url)
    data = response.json()
    print(response.text)
except Exception as e:
    print(e)


{"type": "FeatureCollection","features": [
{"type": "Feature",	"geometry":{"type": "Polygon",	"coordinates": [[[301203.961,2769548.173],[301204.74,2769557.454],[301204.92,2769556.656],[301205.207,2769556.239],[301205.59,2769555.949],[301206.141,2769555.775],[301209.514,2769555.569],[301238.977,2769553.774],[301249.004,2769553.395],[301252.521,2769553.262],[301252.446,2769551.339],[301263.372,2769550.941],[301263.846,2769550.985],[301264.294,2769551.049],[301264.682,2769551.218],[301265.124,2769551.647],[301265.384,2769552.073],[301265.545,2769552.873],[301264.467,2769543.782],[301264.327,2769544.769],[301263.761,2769545.534],[301262.859,2769546.253],[301261.617,2769546.625],[301260.801,2769546.738],[301224.493,2769548.659],[301208.77,2769549.457],[301205.723,2769549.612],[301204.753,2769549.36],[301204.144,2769548.816],[301203.961,2769548.173]]] } ,	"properties": {	"BILL_CODE": "10967113556956",	"URGENT_RCV_DATE": "20241104",	"URGENT_START_DATE": "20241125090000",	"URGENT_END_DATE": "2

In [2]:
import requests
from requests import Response
import csv
import pandas as pd
import os
import sqlite3
from pyproj import Proj, transform

In [4]:
'''得到資料裡面的coordinates 第0個欄位是座標屬於的資料'''
result = []

# 遍歷 features 中的每個 JSON 物件
for feature in data["features"]:
    # 提取該 feature 的 geometry["0"] 中的所有座標
    coordinates_list = feature["geometry"]["coordinates"][0]
    
#     # 提取該 feature 的 BILL_CODE
    bill_code = feature["properties"]["BILL_CODE"]
    
    # # 遍歷每個座標，將 BILL_CODE 和座標結合，並加入結果列表
    for coordinates in coordinates_list:
        result.append([bill_code] + coordinates)

# 顯示結果
display(result)

[['10967113556956', 301203.961, 2769548.173],
 ['10967113556956', 301204.74, 2769557.454],
 ['10967113556956', 301204.92, 2769556.656],
 ['10967113556956', 301205.207, 2769556.239],
 ['10967113556956', 301205.59, 2769555.949],
 ['10967113556956', 301206.141, 2769555.775],
 ['10967113556956', 301209.514, 2769555.569],
 ['10967113556956', 301238.977, 2769553.774],
 ['10967113556956', 301249.004, 2769553.395],
 ['10967113556956', 301252.521, 2769553.262],
 ['10967113556956', 301252.446, 2769551.339],
 ['10967113556956', 301263.372, 2769550.941],
 ['10967113556956', 301263.846, 2769550.985],
 ['10967113556956', 301264.294, 2769551.049],
 ['10967113556956', 301264.682, 2769551.218],
 ['10967113556956', 301265.124, 2769551.647],
 ['10967113556956', 301265.384, 2769552.073],
 ['10967113556956', 301265.545, 2769552.873],
 ['10967113556956', 301264.467, 2769543.782],
 ['10967113556956', 301264.327, 2769544.769],
 ['10967113556956', 301263.761, 2769545.534],
 ['10967113556956', 301262.859, 27695

In [5]:
coordinates_dict = {}

for item in result:
    # 取出每筆資料的編號（例如 '10967113574169'）
    identifier = item[0]
    # 取得對應的坐標資料 (x, y)
    coordinates = [item[1], item[2]]
    
    # 如果這個編號已經在字典裡，則將新的坐標資料加到現有的列表中
    if identifier not in coordinates_dict:
        coordinates_dict[identifier] = []  # 如果該編號還沒有資料，創建一個空列表
    coordinates_dict[identifier].append(coordinates)

display(coordinates_dict)

###############################################################################

# 定義 TWD97 和 WGS84 座標系統
twd97 = Proj(init='epsg:3826')  # TWD97
wgs84 = Proj(init='epsg:4326')  # WGS84

# 假設有多筆 TWD97 坐標 [[x1, y1], [x2, y2], ...]

conn = sqlite3.connect('TPEroad.db')
cursor = conn.cursor()


# 轉換所有 TWD97 坐標為 WGS84
coordinates_wgs84 = []
for key, value in coordinates_dict.items():
    coordinates_twd97 = value
    keyy = key
    for x, y in value:
        # 轉換為 WGS84 坐標
        longitude, latitude = transform(twd97, wgs84, x, y)
        coordinates_wgs84.append([longitude, latitude])
        # display(coordinates_wgs84)
        sql = '''
        INSERT INTO coordinates (BILL_CODE, lat, lon)
        SELECT ?, ROUND(?, 6), ROUND(?, 6)
        WHERE NOT EXISTS (
            SELECT 1 FROM coordinates WHERE lat = ROUND(?, 6)
        );
        '''
        cursor.execute(sql, (keyy, latitude, longitude, latitude))

conn.commit()
cursor.close()
conn.close()


{'10967113556956': [[301203.961, 2769548.173],
  [301204.74, 2769557.454],
  [301204.92, 2769556.656],
  [301205.207, 2769556.239],
  [301205.59, 2769555.949],
  [301206.141, 2769555.775],
  [301209.514, 2769555.569],
  [301238.977, 2769553.774],
  [301249.004, 2769553.395],
  [301252.521, 2769553.262],
  [301252.446, 2769551.339],
  [301263.372, 2769550.941],
  [301263.846, 2769550.985],
  [301264.294, 2769551.049],
  [301264.682, 2769551.218],
  [301265.124, 2769551.647],
  [301265.384, 2769552.073],
  [301265.545, 2769552.873],
  [301264.467, 2769543.782],
  [301264.327, 2769544.769],
  [301263.761, 2769545.534],
  [301262.859, 2769546.253],
  [301261.617, 2769546.625],
  [301260.801, 2769546.738],
  [301224.493, 2769548.659],
  [301208.77, 2769549.457],
  [301205.723, 2769549.612],
  [301204.753, 2769549.36],
  [301204.144, 2769548.816],
  [301203.961, 2769548.173],
  [301203.961, 2769548.173],
  [301204.74, 2769557.454],
  [301204.92, 2769556.656],
  [301205.207, 2769556.239],
  [

c:\Users\user\miniconda3\envs\venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\user\miniconda3\envs\venv\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\user\AppData\Local\Temp\ipykernel_3948\755166745.py:35: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj

In [12]:

# 定義 TWD97 和 WGS84 座標系統
twd97 = Proj(init='epsg:3826')  # TWD97
wgs84 = Proj(init='epsg:4326')  # WGS84

# 假設有多筆 TWD97 坐標 [[x1, y1], [x2, y2], ...]
coordinates_twd97 = coordinates_dict['10967113546859']

# 轉換所有 TWD97 坐標為 WGS84
coordinates_wgs84 = []

for x, y in coordinates_twd97:
    # 轉換為 WGS84 坐標
    latitude, longitude = transform(twd97, wgs84, x, y)
    coordinates_wgs84.append([longitude, latitude])

# 輸出轉換後的所有 WGS84 坐標
for i, (lon, lat) in enumerate(coordinates_wgs84):
    print(f"Point {i+1}: Latitude={round(lon,6)}, Longitude={round(lat,6)}")


c:\Users\ASUS\miniconda3\envs\venv1\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\ASUS\miniconda3\envs\venv1\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_8844\1938517714.py:13: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyp

Point 1: Latitude=25.042934, Longitude=121.50866
Point 2: Latitude=25.042358, Longitude=121.508495
Point 3: Latitude=25.042336, Longitude=121.508579
Point 4: Latitude=25.042377, Longitude=121.50859
Point 5: Latitude=25.042382, Longitude=121.508593
Point 6: Latitude=25.042426, Longitude=121.508628
Point 7: Latitude=25.042431, Longitude=121.50863
Point 8: Latitude=25.042444, Longitude=121.508633
Point 9: Latitude=25.042585, Longitude=121.508673
Point 10: Latitude=25.042696, Longitude=121.508704
Point 11: Latitude=25.042728, Longitude=121.508701
Point 12: Latitude=25.042747, Longitude=121.508699
Point 13: Latitude=25.042756, Longitude=121.508698
Point 14: Latitude=25.04276, Longitude=121.508699
Point 15: Latitude=25.042773, Longitude=121.508702
Point 16: Latitude=25.042846, Longitude=121.508724
Point 17: Latitude=25.042915, Longitude=121.508745
Point 18: Latitude=25.042934, Longitude=121.50866
Point 19: Latitude=25.042934, Longitude=121.50866
Point 20: Latitude=25.042996, Longitude=121.50

In [ ]:
from pyproj import Transformer

# 創建轉換器
transformer = Transformer.from_crs("epsg:3826", "epsg:4326")
twd97_x = 305689.109
twd97_y = 2768207.525
# 轉換 TWD97 到 WGS84
latitude,longitude = transformer.transform(twd97_x, twd97_y)

print(f'TWD97轉WGS84: Latitude: {latitude:.7}, Longitude: {longitude:.7}')


TWD97轉WGS84: Latitude: 25.02091, Longitude: 121.5518


twd97是台灣的座標系統
WGS84是世界的座標系統
經度 longitude
緯度 latitude
所以台灣的X = longitude, Y = latitude

In [14]:
from geopy.geocoders import Nominatim

# 初始化 Nominatim 反向地理編碼器
geolocator = Nominatim(user_agent='PROJ')

def get_address_from_coordinates(lat, lon):
    """ 根據經緯度獲取地址或道路名稱 """
    location = geolocator.reverse((lat, lon), language='zh-TW', exactly_one=True)
    if location:
        return location.address
    return "未找到地址"

# 範例：查詢經緯度 (台北市)
latitude = 25.02066
longitude = 121.5518

address = get_address_from_coordinates(latitude, longitude)
print(address)
print(type(address))
print(f"座標 ({latitude}, {longitude}) 對應的地址是：{address}")


7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣
<class 'str'>
座標 (25.02066, 121.5518) 對應的地址是：7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣


In [47]:
address = "7號, 樂業街118巷, 芳和里, 大安區, 六張犁, 臺北市, 106, 臺灣"
list_address = [item.strip() for item in address.split(",")]
list_address.reverse()
print(list_address)
dic_address = {} 


['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']


In [3]:
    
def download_data():
    '''
    下載資料 需要做驗證只寫入不在資料庫裏面的那筆
    '''
    conn = sqlite3.connect("./TPEroad.db")
    url = 'https://tpnco.blob.core.windows.net/blobfs/Rally/TodayUrgentCase.json'
    try:
        response = requests.get(url)
        response.raise_for_status()
        with conn:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS records (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                Bill_code TEXT,
                RCVdate TEXT,
                Start_date TEXT,
                End_date TEXT,
                Address1 TEXT,
                X1 TEXT,
                Y1 TEXT,
                新地址 TEXT,
                行政區 TEXT,
                Lat TEXT,
                Lon TEXT,
                申請時間 TEXT CHECK (申請時間 LIKE '____-__-__'),
                開始時間 TEXT CHECK (開始時間 LIKE '____-__-__'),
                束時間 TEXT CHECK (結束時間 LIKE '____-__-__')
                UNIQUE(Address1,RCVdate)
            )
            ''')
            print("Table 'records' created or already exists.")
    except Exception as e:
        print(e)

    else:
        with conn:
            cursor = conn.cursor()
            for i in data['features']:
                for k in i['properties']:
                    Bill_code:str = i['properties']["BILL_CODE"]
                    RCVdate = i['properties']["URGENT_RCV_DATE"]
                    Start_date = i['properties']["URGENT_START_DATE"]
                    End_date = i['properties']["URGENT_END_DATE"]
                    Address1 = i['properties']["URGENT_ADDRESS1"]
                    X1 = i['properties']["X1"]
                    Y1 = i['properties']["Y1"]
                    formatted_RCVdate = f"{RCVdate[:4]}-{RCVdate[4:6]}-{RCVdate[6:8]}" if RCVdate else None
                    formatted_Start_date = f"{Start_date[:4]}-{Start_date[4:6]}-{Start_date[6:8]}" if Start_date else None
                    formatted_End_date = f"{End_date[:4]}-{End_date[4:6]}-{End_date[6:8]}" if End_date else None


                    
                    cursor = conn.cursor()
                    sql = '''INSERT OR IGNORE INTO records(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1,申請日期, 開始日期, 結束日期)
                                values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                            '''
                    cursor.execute(sql,(Bill_code, RCVdate, Start_date, End_date, Address1, X1, Y1,formatted_RCVdate, formatted_Start_date, formatted_End_date))
download_data()

near "(": syntax error


現在有每一個x,y轉換成的地址 再把地址存成我用的資料表 再從資料表裡面提取我需要的區的位置 就可以做combobox的下拉式選單

In [3]:
import functoins
import sqlite3

conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()

    sql = '''
        SELECT Address1,x1,y1 FROM records
    '''

    cursor.execute(sql)
    # 把地址跟x1y1座標存進一個字典裡面
    address = [ ]
    for i in cursor.fetchall():
        di = {}
        di["地址"] = i[0]
        di["x1"] = i[1]
        di["y1"] = i[2]
        address.append(di)
    cursor.close() #把cursor關掉 清出記憶體
    display(address)

Table 'records' created or already exists.
資料下載，匯入完成


[{'地址': '中正區延平南路１９７號2樓', 'x1': '301203.961', 'y1': '2769548.173'},
 {'地址': '大同區民權西路１０４之１號', 'x1': '302170.827', 'y1': '2772838.255'},
 {'地址': '松山區八德路三段2號前之部分外側道路及人行道', 'x1': '305460.997', 'y1': '2771219.761'},
 {'地址': '南港區經貿二路１３５號', 'x1': '312274.494', 'y1': '2772584.208'},
 {'地址': '南港區經貿二路１１１號', 'x1': '312251.435', 'y1': '2772354.91'},
 {'地址': '臺北市中山區中山北路 39巷3號(CHANEL專櫃門口)',
  'x1': '302834.061',
  'y1': '2771857.743'},
 {'地址': '中山區林森北路６７巷２４弄（靠近華山公園）', 'x1': '302999.281', 'y1': '2771211.644'},
 {'地址': '大安區復興南路一段１９１之２號', 'x1': '304931.441', 'y1': '2770300.419'},
 {'地址': '中正區忠孝東路一段86號至紹興南街側', 'x1': '302963.774', 'y1': '2770737.324'},
 {'地址': '萬華區昆明街322號', 'x1': '300796.69', 'y1': '2769838.515'},
 {'地址': '萬華區漢中街３４號', 'x1': '301224.665', 'y1': '2770862.247'},
 {'地址': '大安區文昌街210巷9號1樓', 'x1': '305985.878', 'y1': '2769438.269'},
 {'地址': '中正區博愛路２２４巷 自門牌廣州街4巷5弄22號至門牌博愛路224巷2弄2號',
  'x1': '301474.332',
  'y1': '2769639.669'},
 {'地址': '中正區博愛路２１８巷 自門牌218巷1號至224巷2弄2號',
  'x1': '301476.982',
  'y1'

接下來要提取每個字典裏面的x1y1然後丟到我的latlonturn裡面轉成地址(['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街', '114號'])  
再for i in list if '區' in i sql='insert ? into records' execute(sql,(i))


In [4]:
import functoins
import sqlite3
import time
conn = sqlite3.connect("TPEroad.db")
cursor = conn.cursor()
alladdress = []
for i in address:
    ad = i['地址']
    fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
    fullstreet = ''.join(fuad)
    lat,lon = functoins.xytransform(i['x1'],i['y1'])
    for i in fuad:
        if "區" in i :dist = i
    add_content_sql = '''
    UPDATE RECORDS
    SET 新地址 = ?, 行政區 = ?, Lat = ?, Lon = ?
    WHERE address1 = ?;
    '''

    format_date='''
    UPDATE records 
	SET 日期=  (SELECT strftime('%Y-%m-%d', 
	              substr(RCVdate , 1, 4) || '-' || substr(RCVdate , 5, 2) || '-' || substr(RCVdate , 7, 2)) )
    '''


    cursor.execute(add_content_sql,(fullstreet, dist, lat, lon, ad))
    time.sleep(10)
    conn.commit()


1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.03318, Longitude: 121.5074
3.單次轉換結果['臺灣', '100', '臺北市', '龍匣口', '中正區', '南門里', '延平南路195巷', '2號']
4.總清單:[['臺灣', '100', '臺北市', '龍匣口', '中正區', '南門里', '延平南路195巷', '2號']]
2.TWD97轉WGS84: Latitude: 25.03318, Longitude: 121.5074


KeyboardInterrupt: 

In [6]:
import functoins
address = [{'地址': '台北市樂業街108巷1至11號', 'x1': '305689.109', 'y1': '2768179.406'},
 {'地址': '士林區大東路１７６號', 'x1': '302989.777', 'y1': '2776097.833'}]
alladdress = []
# for i in address:
#     ad = i['地址']
#     fuad = functoins.latlonturn("reverse",alladdress,i['x1'],i['y1'])
#     type(fuad)
# for i in fuad:
#     if "區" in i :dist = i
#     print(dist)

fuad = functoins.latlonturn('reverse', alladdress, address[0]["x1"], address[0]["y1"])
fullstreet = ''.join(fuad)
display(fullstreet)

1.執行反轉操作
2.TWD97轉WGS84: Latitude: 25.02066, Longitude: 121.5518
3.單次轉換結果['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']
4.總清單:[['臺灣', '106', '臺北市', '六張犁', '大安區', '芳和里', '樂業街118巷', '7號']]


'臺灣106臺北市六張犁大安區芳和里樂業街118巷7號'

In [4]:
import sqlite3
def get_district()->list[str]:
    '''
    docString
    parameter:
    return:
        傳出所有的行政區名稱
    '''
    conn = sqlite3.connect("TPEroad.db")
    with conn:
        # Create a cursor object to execute SQL commands
        cursor = conn.cursor()
        # SQL query to select unique sitenames from records table
        sql = '''
        SELECT DISTINCT 行政區
        FROM records
        '''
        # Execute the SQL query
        cursor.execute(sql)
        # Get all results and extract first item from each row into a list
        counties = [items[0] for items in cursor.fetchall()]
    
    # Return the list of unique sitenames
    return counties
get_district()

['北投區',
 '大安區',
 '松山區',
 '士林區',
 '內湖區',
 '文山區',
 '南港區',
 '中山區',
 '大同區',
 '萬華區',
 '中正區',
 '信義區']

從資料庫拿coordinates的座標畫在mapview上面

In [3]:
bill_code = '10967113575858'
conn = sqlite3.connect('TPEroad.db')
with conn:
    cursor = conn.cursor()
    sql = '''
        SELECT distinct lat,lon
        FROM coordinates_test
        where BILL_CODE =?
        '''
    # Execute the SQL query
    cursor.execute(sql, (bill_code,))

    result = cursor.fetchall()
    coordinate_list = [[lat, lon] for lat, lon in result]


    print(coordinate_list)
    

[[25.041117, 121.52214], [25.041047, 121.522122], [25.041882, 121.518678], [25.041965, 121.518709], [25.041947, 121.518717], [25.041938, 121.518725], [25.041933, 121.51874], [25.041837, 121.519107], [25.041804, 121.519247], [25.041846, 121.519262], [25.041831, 121.519326], [25.041789, 121.519314], [25.041745, 121.519521], [25.041786, 121.519533], [25.04177, 121.519595], [25.041732, 121.519586], [25.04173, 121.519586], [25.041726, 121.519588], [25.04151, 121.520504], [25.041544, 121.520513], [25.041524, 121.520601], [25.041489, 121.520591], [25.041307, 121.521339], [25.041121, 121.522117], [25.04098, 121.522105], [25.041063, 121.521767], [25.041225, 121.521088], [25.041381, 121.520435], [25.041426, 121.520245], [25.041516, 121.51987], [25.041538, 121.519785], [25.041659, 121.519291], [25.041679, 121.51921], [25.041727, 121.519013], [25.041686, 121.519003], [25.041719, 121.518861], [25.041758, 121.518873], [25.041798, 121.518713], [25.0418, 121.51869], [25.041799, 121.518677], [25.041797

tkmap做兩點距離或範圍

In [ ]:
import tkinter as tk
from tkinter import ttk
import tkintermapview

# 創建 Tkinter 主窗口
root = tk.Tk()
root.title("./tkintermapview Example")

# 設置地圖視窗
map_frame = ttk.Frame(root)
map_widget = tkintermapview.TkinterMapView(map_frame, width=600, height=400, corner_radius=0)

# 設定初始地圖顯示位置（台北市）
# map_widget.set_position(25.02091606, 121.55139558, marker=True)
map_widget.set_position(25.042934, 121.50866)

# # 三個點的經緯度
points = [
    {"name": "Point 1", "lat": 25.0330, "lon": 121.5674},  # 例如台北101
    {"name": "Point 2", "lat": 25.0340, "lon": 121.5644},  # 點2
    {"name": "Point 3", "lat": 25.0320, "lon": 121.5664},  # 點3
]
ppoint = coordinates_wgs84







# # 在地圖上添加這三個點的標記
# for point in ppoint:
#     map_widget.set_marker(point[0], point[1])

# # 計算這三個點的範圍（bounding box）
# lats = [point["lat"] for point in points]
# lons = [point["lon"] for point in points]
# min_lat, max_lat = min(lats), max(lats)
# min_lon, max_lon = min(lons), max(lons)

# # 計算範圍的中心點和放大等級
# center_lat = (min_lat + max_lat) / 2
# center_lon = (min_lon + max_lon) / 2

# # 設置地圖顯示範圍
# map_widget.set_position(center_lat, center_lon, zoom=15)

# # 畫出範圍的矩形
map_widget.set_polygon(ppoint)

# 打包顯示地圖
map_widget.pack()
map_frame.pack(padx=10, pady=10)

# 啟動 Tkinter 主循環
root.mainloop()


In [10]:
import sqlite3
ad = '臺灣11218臺北市奇岩北投區永明里'
def get_data():
    conn = sqlite3.connect('./TPEroad.db')
    with conn:
        cursor = conn.cursor()        
        sql = '''
        SELECT c.Bill_code, c.Lat, c.Lon
        FROM coordinates c
        JOIN records r ON c.Bill_code = r.Bill_code
        WHERE r.新地址 = ?;
        '''
        cursor.execute(sql,(ad,))
        address_list = [list(item) for item in cursor.fetchall()]
                # 检查返回的结果
        if address_list:
            print(address_list)
        else:
            print("No data found for the given address.")
get_data()

[['10967113569840', '25.118111', '121.517445'], ['10967113569840', '25.118113', '121.517448'], ['10967113569840', '25.118119', '121.517453'], ['10967113569840', '25.118122', '121.517454'], ['10967113569840', '25.118145', '121.517506'], ['10967113569840', '25.118152', '121.517454'], ['10967113569840', '25.118175', '121.51756'], ['10967113569840', '25.118176', '121.517563'], ['10967113569840', '25.118181', '121.517533'], ['10967113569840', '25.118185', '121.517525'], ['10967113569840', '25.118189', '121.51752'], ['10967113569840', '25.11829', '121.517457'], ['10967113569840', '25.118429', '121.51733'], ['10967113569840', '25.118433', '121.517364'], ['10967113569840', '25.118455', '121.517259'], ['10967113569840', '25.118457', '121.517257'], ['10967113569840', '25.118462', '121.517254'], ['10967113569840', '25.118467', '121.517249'], ['10967113569840', '25.118472', '121.517245'], ['10967113569840', '25.118477', '121.51724'], ['10967113569840', '25.118481', '121.517235'], ['10967113569840'

In [6]:
import tkinter as tk
from tkinter import ttk
from tkintermapview import TkinterMapView

# 假設這是你所提供的亂序座標列表
coordinates = [[25.041117, 121.52214], [25.041047, 121.522122], [25.041882, 121.518678], [25.041965, 121.518709], [25.041947, 121.518717], [25.041938, 121.518725], [25.041933, 121.51874], [25.041837, 121.519107], [25.041804, 121.519247], [25.041846, 121.519262], [25.041831, 121.519326], [25.041789, 121.519314], [25.041745, 121.519521], [25.041786, 121.519533], [25.04177, 121.519595], [25.041732, 121.519586], [25.04173, 121.519586], [25.041726, 121.519588], [25.04151, 121.520504], [25.041544, 121.520513], [25.041524, 121.520601], [25.041489, 121.520591], [25.041307, 121.521339], [25.041121, 121.522117], [25.04098, 121.522105], [25.041063, 121.521767], [25.041225, 121.521088], [25.041381, 121.520435], [25.041426, 121.520245], [25.041516, 121.51987], [25.041538, 121.519785], [25.041659, 121.519291], [25.041679, 121.51921], [25.041727, 121.519013], [25.041686, 121.519003], [25.041719, 121.518861], [25.041758, 121.518873], [25.041798, 121.518713], [25.0418, 121.51869], [25.041799, 121.518677], [25.041797, 121.518665], [25.041769, 121.518639], [25.04175, 121.518629]]


# 根據經度和緯度來排序座標
sorted_coordinates = sorted(coordinates, key=lambda x: (x[0], x[1]))

# 確定四個角點
x_values = [coord[0] for coord in sorted_coordinates]
y_values = [coord[1] for coord in sorted_coordinates]

# 確定長方形的四個角點
left_bottom = (min(x_values), min(y_values))
right_bottom = (max(x_values), min(y_values))
left_top = (min(x_values), max(y_values))
right_top = (max(x_values), max(y_values))

# 用正確的順序繪製長方形的四個角
ordered_coordinates = [left_bottom, left_top, right_top, right_bottom, left_bottom]

# 創建Tkinter視窗
root = ttk.Frame()
map_widget = TkinterMapView(root, width=800, height=600)
map_widget.pack(fill=tk.BOTH, expand=True)
map_widget.set_position(25.041797, 121.518665)
# 設定地圖的中心位置
map_widget.set_position(3, 2)

# 在地圖上畫出四個角並連線成長方形
for coord in ordered_coordinates:
    map_widget.set_marker(coord[0], coord[1])

# 繪製長方形框架
map_widget.set_polygon(ordered_coordinates)
root.pack(padx=10,pady=10)
# 顯示地圖
root.mainloop()
